In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=18ff71da8a275f88aafb6b06b7c632f623d9392d8894f9b2938da5609b5bf708
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


# Load data from Excel file


In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

hc = pd.read_csv('/content/drive/MyDrive/hcV3-stories.csv', sep=',', encoding = "ISO-8859-1")


Mounted at /content/drive


In [ ]:
hc.head()

,AssignmentId,WorkTimeInSeconds,WorkerId,annotatorAge,annotatorGender,annotatorRace,distracted,draining,frequency,importance,...,mostSurprising,openness,recAgnPairId,recImgPairId,similarity,similarityReason,story,stressful,summary,timeSinceEvent
0,32RIADZISTQWI5XIVG5BN0VMYFRS4U,1641,E9TY34YY,25.0,man,white,1.0,1.0,NaN,3.0,...,when I got concert tickets,0.00,NaN,3018Q3ZVOJCZJFDMPSFXATCQ4DARA2,3.0,"I've been to a couple concerts, but not many.","Concerts are my most favorite thing, and my bo...",1.0,My boyfriend and I went to a concert together ...,90.0
1,3018Q3ZVOJCZJFDMPSFXATCQ4DARA2,1245,237K2NI1,25.0,woman,white,1.0,1.0,3.0,4.0,...,we saw the beautiful sky.,1.00,NaN,3018Q3ZVOJCZJFDMPSFXATCQ4DARA2,NaN,NaN,"The day started perfectly, with a great drive ...",1.0,My boyfriend and I went to a concert together ...,90.0
2,3IRIK4HM3B6UQBC0HI8Q5TBJZLEC61,1159,FK5QTANB,35.0,woman,black,1.0,1.0,NaN,4.0,...,she went into labor early,0.50,NaN,3018Q3ZVOJCZJFDMPSFXATCQG04RAI,3.0,I am a mother myself,It seems just like yesterday but today makes f...,1.0,My sister gave birth to my twin niece and neph...,150.0
3,3018Q3ZVOJCZJFDMPSFXATCQG04RAI,500,UYOSBBRS,30.0,woman,white,1.0,4.0,3.0,5.0,...,finding out they were healthy.,1.00,NaN,3018Q3ZVOJCZJFDMPSFXATCQG04RAI,NaN,NaN,"Five months ago, my niece and nephew were born...",2.0,My sister gave birth to my twin niece and neph...,150.0
4,3MTMREQS4W44RBU8OMP3XSK8NMJAWZ,1074,34BFLNJV,25.0,man,white,2.0,2.0,NaN,3.0,...,When I don't answer the phone in case I owe th...,0.25,NaN,3018Q3ZVOJCZJFDMPSFXATCQG06AR3,4.0,Because I also have money problems,About a month ago I went to burning man. I was...,4.0,It is always a journey for me to go to burning...,30.0


Here, we change values of memType to numeric and change the names of story and memType to sent and labels.

In [ ]:
mem = hc[hc['memType']!='retold']
mem = mem.dropna(subset=['story', 'memType'])
mem = mem[['story', 'memType']]
mem['memType'][mem['memType']=='recalled']= 0
mem['memType'][mem['memType']=='imagined']= 1
mem = mem.rename(columns={'story': 'sent', 'memType': 'labels'})
mem['type']='B'

In [ ]:
mem.head()

,sent,labels,type
0,"Concerts are my most favorite thing, and my bo...",1,B
1,"The day started perfectly, with a great drive ...",0,B
2,It seems just like yesterday but today makes f...,1,B
3,"Five months ago, my niece and nephew were born...",0,B
4,About a month ago I went to burning man. I was...,1,B


# Tokenizer and Model


In [ ]:
class CustomDataset(Dataset):
    def init(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def len(self):
        return len(self.data)

    def getitem(self, idx):
        sample = {'data': self.data[idx], 'labels': self.labels[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Preprocessing


Here we also drop the rows with very short stories (stories with less than 60 words).

In [ ]:
from torch.utils.data import Dataset

# Print columns before cleaning
print("Columns before cleaning:", mem.columns)

# Clean column names
mem.columns = mem.columns.str.strip().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

print("Columns after cleaning:", mem.columns)

# Check the DataFrame structure
print("DataFrame structure:")
print(mem.head())

# Drop rows where any of the specified columns have NaN values
mem = mem.dropna()
#see if there is any NA value existing
mem.isna().sum()
mem.shape[0]

mem['word_count'] = mem['sent'].apply(lambda x: len(str(x).split()))
mem = mem[mem['word_count'] >= 60]

# Drop the 'word_count' column as it was used for filtering
mem = mem.drop(columns=['word_count'])

# Print the resulting DataFrame
print("DataFrame after filtering:")
print(mem.head())
mem.shape[0]

Columns before cleaning: Index(['sent', 'labels', 'type'], dtype='object')
Columns after cleaning: Index(['sent', 'labels', 'type'], dtype='object')
DataFrame structure:
                                                sent labels type
0  Concerts are my most favorite thing, and my bo...      1    B
1  The day started perfectly, with a great drive ...      0    B
2  It seems just like yesterday but today makes f...      1    B
3  Five months ago, my niece and nephew were born...      0    B
4  About a month ago I went to burning man. I was...      1    B
DataFrame after filtering:
                                                sent labels type
0  Concerts are my most favorite thing, and my bo...      1    B
1  The day started perfectly, with a great drive ...      0    B
2  It seems just like yesterday but today makes f...      1    B
3  Five months ago, my niece and nephew were born...      0    B
4  About a month ago I went to burning man. I was...      1    B


5450

In [ ]:
def create_dataset(Scenario={'train': [], 'test': [], 'scenario': 1}, mem=None, seed=42, cv=5):
    train = {f'split_{i + 1}': [] for i in range(cv)}
    test = {f'split_{i + 1}': [] for i in range(cv)}

    np.random.seed(seed)

    num_row = (mem.index.shape[0] - (mem.shape[0] % 10))
    me_arr = np.random.choice(mem.index.shape[0], int(num_row), replace=False)
    me_arr_r = me_arr.reshape((cv, int(num_row / cv)))

    cv_ind = 0
    for k in train.keys():
        if 'hyppocorpus' in Scenario['train']:
            ind = me_arr_r[cv_ind, :]
            antind = [i for i in me_arr if i not in ind]
            mem_train = mem[mem.index.isin(antind)]
            mem_test = mem[mem.index.isin(ind)]

            train[k].append(mem_train)
            test[k].append(mem_test)

        train[k] = pd.concat(train[k])
        test[k] = pd.concat(test[k])

        cv_ind += 1

    #  return should be outside the loop
    return train, test


In [ ]:
# Funzione di preprocessamento per il tuo dataset
def preprocess_function(examples):
    return tokenizer(examples["sent"], truncation=True, padding=True)

#Experiment Scenario 1
# model


Here we used GPT2ForSequenceClassification to do classification with GPT2.

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm


# Configurazione iniziale
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
numcv = 5
num_epochs = 3
model_size = 'gpt2'
Scenario = {'train': ['hyppocorpus'], 'test': ['hyppocorpus'], 'scenario': 1}

# Caricamento e preparazione dei dati
train, test = create_dataset(Scenario, mem, seed, numcv)

# Inizializzazione del tokenizer e del modello
tokenizer = GPT2Tokenizer.from_pretrained(model_size)
tokenizer.pad_token = tokenizer.eos_token  # Imposta il token di padding
model = GPT2ForSequenceClassification.from_pretrained(model_size, num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id  # Aggiorna il modello con il token di padding

results = {}

for sp in tqdm(train.keys()):
    if not isinstance(train[sp], pd.DataFrame):
        print(f"Error: train[{sp}] is not a DataFrame. It's a {type(train[sp])}.")
        continue
    # Preparazione del dataset
    data_train = Dataset.from_pandas(train[sp]).map(preprocess_function, batched=True)
    data_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    data_test = Dataset.from_pandas(test[sp]).map(preprocess_function, batched=True)
    data_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    # Configurazione degli argomenti di training
    training_args = TrainingArguments(
        output_dir=f"{model_size}_scenario_{Scenario['scenario']}",
        evaluation_strategy='epoch',
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        fp16=False,
        push_to_hub=False,
    )

    # Inizializzazione del Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_test,
        tokenizer=tokenizer
    )

    # Training del modello
    trainer.train()

    # Funzione per calcolare le previsioni
    def compute_predictions(model, data_loader):
        model.eval()
        predictions = []
        actuals = []

        with torch.no_grad():
          for batch in data_loader:
            # Ora 'batch' è un dizionario
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)
            labels = batch['labels'].to(model.device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            predictions.extend(preds.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

          return predictions, actuals

    # Preparazione del DataLoader di test
    test_loader = DataLoader(data_test, batch_size=2)

    # Calcolo delle previsioni
    predictions, actual_labels = compute_predictions(model, test_loader)

    # Calcolo dell'accuratezza
    accuracy = accuracy_score(actual_labels, predictions)
    print(f'Accuracy for {sp}: {accuracy}')

    # Salvataggio dei risultati
    results[sp] = accuracy

# Calcolo dell'accuratezza media
average_accuracy = np.mean(list(results.values()))
print('Average accuracy:', average_accuracy)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

Map:   0%|          | 0/3435 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.414500,0.113312


Epoch,Training Loss,Validation Loss
1,0.414500,0.113312
2,0.249000,0.117817
3,0.121800,0.152929


Accuracy for split_5: 0.9805045871559633
Average accuracy: 0.8967397600581588
